In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr

# 1. Prepare Data

## 1.1 Load

In [ ]:
%%time
train = pd.read_parquet('../input/ubiquant-parquet/train_low_mem.parquet')
#display(train.info(), train.head())

## 1.2 Prepare Testing Dataframes

In [ ]:
features = [f'f_{i}' for i in range(300)]

X = train[features]
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y)

#display(X_train, X_test, y_train, y_test)

In [ ]:
# prepare dataframes for model fitting

# dataframe is too huge - select just first N features and M lines of dataframe
N = 100
M = 1000000
features = [f'f_{i}' for i in range(N)]

X_train_reduced = train[features][:M]
y_train_reduced = train['target'][:M]
display(X_train_reduced, y_train_reduced)

# 2. Create Model - LGBM

In [ ]:
%%time
import lightgbm as ltb

model_ltb = ltb.LGBMRegressor()
model_ltb.fit(X_train_reduced, y_train_reduced)
print(); print(model_ltb)

In [ ]:
def predictLGB(prediction_df):
    X_predict = prediction_df[X_train_reduced.columns]
    prediction = model_ltb.predict(X_predict)
    return prediction

In [ ]:
%%time
predLGB = predictLGB(X_test)

predLGB

In [ ]:
# evaluate
pearsonr(y_test, predLGB)[0]

# 3. Submission

In [ ]:
import ubiquant
env = ubiquant.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
for (test_df, sample_prediction_df) in iter_test:
    sample_prediction_df['target'] = predictLGB(test_df)  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions